# The Eindhoven datagrid

Notebook to construct the Eindhoven Grid Dataset. 
Main sections:
1. Function definitions
1. Action! the actual script


#### Imports, constants, settings

In [29]:
# IMPORTS
import math
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import shape,mapping
import json
import statistics as stat
import warnings
warnings.filterwarnings("ignore")

# GRID OUTLINE and RESOLUTION 
GRID_TOP_LAT = 51.500
GRID_LEFT_LON = 5.350
GRID_BOT_LAT = 51.400
GRID_RIGHT_LON = 5.550
TILESIZE_LAT = 0.001
TILESIZE_LON = 0.001

## 1. FUNCTIONS 

#### 1.1 Initial empty grid

In [30]:
# generate the initial grid, with "empty" tiles
# Latitude and Longitude are represented as integers, that is * 1000
# In: just the settings
# Out: empty grid
def createEmptyGrid():
    lat_list = []
    lon_list = []

#    for lat in range(int(GRID_BOT_LAT*1000),int(GRID_TOP_LAT*1000),int(TILESIZE_LAT*1000)):
#        for lon in range(int(GRID_LEFT_LON*1000),int(GRID_RIGHT_LON*1000),int(TILESIZE_LON*1000)):
    for lat in range(int(GRID_BOT_LAT*1000),int(GRID_TOP_LAT*1000),int(TILESIZE_LAT*1000)):
        for lon in range(int(GRID_LEFT_LON*1000),int(GRID_RIGHT_LON*1000),int(TILESIZE_LON*1000)):

            lat_list.append(lat)
            lon_list.append(lon)

    df = pd.DataFrame({'lat': lat_list, 'lon': lon_list})
    return df

# floor a float to 3 decimals
def floor3(x):
    return (np.floor(x*1000)/1000)

In [31]:
def distance_in_meters(lat1, lon1, lat2, lon2):
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000; # meters

print("0.001 lng difference means {} meters".format(distance_in_meters(51.5,5.300,51.5,5.301)))
print("0.001 lon difference means {} meters".format(distance_in_meters(51.400,5.3,51.401,5.3)))
print("0.01 lng difference means {} meters".format(distance_in_meters(51.4,5.30,51.4,5.31)))
print("0.01 lon difference means {} meters".format(distance_in_meters(51.40,5.3,51.41,5.3)))

0.001 lng difference means 69.29801236136062 meters
0.001 lon difference means 111.31949079369154 meters
0.01 lng difference means 694.4995896816404 meters
0.01 lon difference means 1113.1949079319584 meters


#### 1.2 Neighbourhoods / Buurten info

In [32]:
# add buurtcode + buurt to all the tiles intersecting this buurt (buurtshape)
# In: buurtcode, buurtshape, df[lat,lon,Buurtcode,BuurtIA]
# Out: df[Buurtcode,BuurtIA] extended with the intersection areas of this current buurt with the tile collection
def findTilesInBuurt(df, buurtcode, buurtshape):
    min_lon, min_lat, max_lon, max_lat = buurtshape.bounds
    min_lon = floor3(max(min_lon-TILESIZE_LON,GRID_LEFT_LON))
    min_lat = floor3(max(min_lat-TILESIZE_LAT,GRID_BOT_LAT))
    max_lon = floor3(min(max_lon+TILESIZE_LON,GRID_RIGHT_LON))
    max_lat = floor3(min(max_lat+TILESIZE_LAT,GRID_TOP_LAT))

    for lat in np.arange(min_lat, max_lat, TILESIZE_LAT):
        for lon in np.arange(min_lon,max_lon, TILESIZE_LON):
            tile = Polygon([(lon,lat), (lon,lat+TILESIZE_LAT), (lon+TILESIZE_LON,lat+TILESIZE_LAT), (lon+TILESIZE_LON,lat)])
            try:
                int_area = buurtshape.intersection(tile).area
                if int_area > 0:
                    df.loc[(int(1000*lat), int(1000*lon)),'Buurtcode'].append(buurtcode)
                    df.loc[(int(1000*lat), int(1000*lon)),'BuurtIA'].append(int_area)
            except:
                print("{0}: ERR tile ({},{}) intersect shape ({})".format(__name__,lat,lon,buurtshape))
    return 

# assigns each tile to the neighbourhood that intersects it the most
# uses findTilesInBuurt
# In: df[lat,lon] , dfb = buurten information dataframe
# Out: df[Buurtcode, Buurt] 
def addBuurten(df,dfb):
    # make temporary "Buurt" and "BuurtIntersectionArea" columns
    df["Buurtcode"] = 0
    df["Buurtcode"] = df["Buurtcode"].apply(lambda x: [])
    df["BuurtIA"] = 0 
    df["BuurtIA"] = df["BuurtIA"].apply(lambda x: [])
    df.set_index(['lat','lon'],inplace=True)

    # loop through the list of buurten, mark the intersecting tiles
    for i in range(0,len(dfb)):
        bpoly = shape(json.loads(dfb.loc[i,'geo_shape']))
        ### print(dfb.loc[i,'Buurtcode '])
        findTilesInBuurt(df,dfb.loc[i,'Buurtcode '],bpoly)

    df.reset_index(inplace=True)

    # keep the buurt with max intersection area
    df['maxIA'] = [x.index(max(x)) if len(x)>0 else -1 for x in df['BuurtIA']]
    df['Buurtcode'] = df.apply(lambda row: row['Buurtcode'][row['maxIA']] if row['maxIA']>=0 else 0, axis=1)
    df.drop(['BuurtIA','maxIA'],axis=1,inplace=True)
    
    # add the name - not efficient, but..
    df = df.merge(dfb[['Buurtcode ','Buurtnaam ']],left_on="Buurtcode",right_on="Buurtcode ",how="left")
    df['Buurtnaam '].fillna('Buiten', inplace=True)
    ### df['Buurtnaam '] = df['Buurtnaam '].apply(str.capitalize)
    df.rename(columns={'Buurtnaam ': 'Gcb.buurt','Buurtcode': 'Gcb.buurtcode'},inplace=True) 
    df.drop('Buurtcode ',axis=1,inplace=True)
    
    return df


In [33]:
# adds number of inhabitants per tile
# In: df[cbG.buurtcode,lat] , dfpop = population per buurt 
# Out: df[nP.pop] 
def addPopulationDensity(df, dfpop):
    dfg = df.groupby('Gcb.buurtcode')['lat'].count().reset_index()
    dfg.rename(columns={'lat':'ntiles'},inplace=True)
    dfpop.dropna(inplace=True)
    dfpop['Buurtcode '] = dfpop['Buurtcode '].fillna(-1).astype(int)
    dfpop = dfpop.merge (dfg,left_on='Buurtcode ',right_on='Gcb.buurtcode',how="right")
    # dfpop.drop('Buurtcode ',axis=1,inplace=True)
    dfpop['Pnb.pop'] = round(dfpop['number of residents'] / dfpop['ntiles'])
    dfpop.drop(['number of residents','ntiles'],axis=1,inplace=True)
    # compute a density for each tile
    df = df.merge(dfpop,on="Gcb.buurtcode",how="left")
    df.drop('Buurtcode ',axis=1,inplace=True)
    return df

In [34]:
# TODO: keep some of these columns! Electricity Consumption seems to link with AQ :-)
Cols2Drop = ['Buurten', 'aantal woningen',
       'Totaal aantal huishoudens','gemiddeld elektriciteitsverbruik totaal',
       'Doet aan vrijwilligerswerk','Heeft een langdurige ziekte',
       'Ervaart een beperkt sociaal netwerk', 'Middelbaar opleidingsniveau %',
       'Rapportcijfer dat mensen geven aan veiligheid in hun buurt', 'Totaal 0-14 jaar',
       'Totaal 65 jaar en ouder','Totaal 15-64 jaar','Aandeel huishoudens dat gebruik maakt van minimaregeling','Totaal aantal vestigingen',
       'Totaal aantal werkzame personen (inclusief uitzendkrachten)',
       'Totaal aantal winkelpanden', 'Oppervlakte totaal (in ha)',
       'Stedelijkheid']



In [35]:
RenameDict = {'schaalscore etnische diversiteit':'Pnb.diversity', 
       'woz-waarde (x 1000)':'Enb.woz', 
       'Heeft last van vervuilde lucht %':'Pnbp.complains_aq',
       'Heeft last van (minstens 1 vorm van) geluidshinder %':'Pnbp.complains_noise',
       'verplaatst zich meestal per auto %':'Pnbp.mostly_by_car',
       'verplaatst zich meestal te voet %':'Pnbp.mostly_walking', 
       'Voelt zich niet zo gelukkig of ongelukkig':'Pnbp.unhappy',
       'Hoog opleidingsniveau %':'Enbp.high_education',
       'Laag opleidingsniveau %':'Enbp.low_education',
       'Aandeel personen dat de perceptie heeft dat er veel criminaliteit in de eigen buurt is':'Pnbp.feels_unsafe',
       'schaalscore sociale cohesie':'Pnb.cohesion',
       'Rapportcijfer dat inwoners geven aan hoe prettig het wonen is in hun buurt':'Pnb.good_life',
       'Geregistreerde werkzoekenden UWV zonder dienstverband t.o.v. het aantal 15 t/m 74 jarigen':'Enbp.unemployed',
       'Aandeel economisch zelfstandig':'Enbp.independent',
       'Gemiddeld besteedbaar huishoudeninkomen (x1000 euro)':'Enb.avg_income',
       }

def addBuurtStuff(df,dfbi):
    df = df.merge(dfbi, left_on="Gcb.buurt", right_on="Buurten", how="left")
    df=df.drop(Cols2Drop,axis=1)
    df=df.rename(columns=RenameDict)
    return df

#### 1.3 Point info over Environment
* trees registered in the municipality database
* biodiversity observation data

In [36]:
# trees get aggregated in various ways for each tile: 
# total number, median height, number of species, most common species

def addTreeInfo(df, dfb):
   # transform the geo string to lat/lon coordinates - times 1000, to store them as int
   dfb[["lat", "lon"]] = dfb["geo_point_2d"].str.split(",", expand=True).astype(float)
   dfb["lat"] = np.floor(dfb["lat"] * 1000).astype(int)
   dfb["lon"] = np.floor(dfb["lon"] * 1000).astype(int)

   dfb[['Hmin','Hmax']] = dfb['HOOGTE'].str.split('-',expand=True)
   dfb['Hmin'] = dfb['Hmin'].str.replace('>','')
   dfb['Hmin']=dfb['Hmin'].astype('Int64')
   dfb['Hmax']=dfb['Hmax'].astype('Int64')
   dfb['HOOGTE'] = (dfb['Hmin'] + dfb['Hmax'])/2

   dfb.drop(['geo_point_2d','Hmin','Hmax'],axis=1,inplace=True)
   dfb.dropna(inplace=True) # TODO: change this!!

   # aggregate by lat and lon, to get one row per tile
   grouped = dfb.groupby(['lat', 'lon']).agg({'BOOMSOORT': list, 'BOOMSOORT_NEDERLANDS': list, 'HOOGTE':list, 'PLANTJAAR': list}).reset_index()
   grouped['Vn.number_trees'] = [len(l) for l in grouped['BOOMSOORT']]
   grouped['Vn.number_tree_species'] = [len(set(l)) for l in grouped['BOOMSOORT_NEDERLANDS']]
   grouped['Vc.mode_tree_species'] = [stat.mode(l) for l in grouped['BOOMSOORT_NEDERLANDS']]
   ###grouped['Vc.mode_tree_species'] = grouped['Vc.mode_tree_species'].apply(str.capitalize())
   # grouped['boom_hoogte_median'] = [stat.median(l) for l in grouped['HOOGTE']] # zou iets anders moeten doen hier, median is beter
   grouped['Vn.med_tree_height'] = [stat.median(l) for l in grouped['HOOGTE']]
   # grouped['Trees.MedianPlantYear'] = [stat.median(l) for l in grouped['PLANTJAAR']]
   grouped.drop(['BOOMSOORT', 'BOOMSOORT_NEDERLANDS', 'HOOGTE','PLANTJAAR'],axis=1,inplace=True)
    
   # merge met df op lat en lon
   df = pd.merge(df, grouped, on = ['lat','lon'], how='left')
   return df

In [37]:
# Observation data from GBIF
# TODO: ask Waarneming.nl to use their data, is much more

def addBiodiversityInfo(df,dfobs):
    dfobs.rename(columns={'decimalLatitude':'lat', 'decimalLongitude':'lon'},inplace=True)
    dfobs=dfobs[dfobs["year"]>=2020].dropna()
    dfobs=dfobs[dfobs["lat"]>=GRID_BOT_LAT]
    dfobs=dfobs[dfobs["lat"]<=GRID_TOP_LAT]
    dfobs=dfobs[dfobs["lon"]>=GRID_LEFT_LON]
    dfobs=dfobs[dfobs["lon"]<=GRID_RIGHT_LON]
    dfobs["lat"] = np.floor(dfobs["lat"] *1000).astype(int)
    dfobs["lon"] = np.floor(dfobs["lon"] *1000).astype(int)
    # aggregate by lat and lon, to get one row per tile
    grouped = dfobs.groupby(['lat', 'lon']).agg({'species': list}).reset_index()
    grouped['Vn.number_species'] = [len(l) for l in grouped['species']]
    grouped['Vc.mode_species'] = [stat.mode(l) for l in grouped['species']]
    grouped['Vc.mode_species'] = grouped['Vc.mode_species'].apply(str.upper)
    grouped.drop(['species'],axis=1,inplace=True)
    #merge met df op lat en lon
    df = pd.merge(df, grouped, on = ['lat','lon'], how='left')
    print("{0}: ".format(__name__,df.shape))
    df = df.drop_duplicates() # <- why needed?
    return df


#### 1.4 Air Quality Sensor info

In [38]:
# AQ data aggregated in various ways: avg, median, max, min
# first per sensor, then per tile
# TODO: talk to TNO about the best aggregation

def agg_AQ_per_Location(aq):
    aq['Date'] = pd.to_datetime(aq['Date'])
    aq['Time'] = [pd.Timestamp(t) for t in aq['Time']]
    aq['Year'] = aq['Date'].dt.year
    aq['Hour'] = [t.hour for t in aq['Time']]
    aq['day'] = ['day' if (h>=6 and h<=22) else 'night' for h in aq['Hour']]
    aq.drop(['Date','Time','Hour'],axis=1,inplace=True)
    #aq.sample(4)

    #grouped = aq.groupby(['Location','Year','day']).agg({'PM1': list, 'PM2.5': list, 'PM10':list}).reset_index()
    grouped = aq.groupby(['Location','Year']).agg({'PM1': list, 'PM2.5': list, 'PM10':list}).reset_index()
    grouped['PM10_med'] = [stat.median(l) for l in grouped['PM10']]
    grouped['PM10_avg'] = [stat.mean(l) for l in grouped['PM10']]
    grouped['PM10_max'] = [max(l) for l in grouped['PM10']]
    grouped['PM10_min'] = [min(l) for l in grouped['PM10']]
    grouped['PM10_range'] = grouped['PM10_max'] - grouped['PM10_min']

    grouped.drop(['PM1', 'PM2.5', 'PM10'],axis=1,inplace=True)

    aqlocs = pd.DataFrame(columns=['LocationID'])
    aqlocs['LocationID']=grouped['Location'].unique()
    aqlocs.set_index('LocationID',inplace=True)

    #
    grouped.set_index(['Location','Year'],inplace=True)
    for i in grouped.index:
        colname = "Vn.PM10_med_" + str(i[1]) 
        aqlocs.loc[i[0],colname] = grouped.loc[i,'PM10_med']
        colname = "Vn.PM10_avg_" + str(i[1]) 
        aqlocs.loc[i[0],colname] = grouped.loc[i,'PM10_avg']
        colname = "Vn.PM10_max_" + str(i[1]) 
        aqlocs.loc[i[0],colname] = grouped.loc[i,'PM10_max']
    #
    aqlocs.reset_index(inplace=True)
    return aqlocs

def agg_AQ_per_Tile(df,aqlocs,locs):
    locs=locs[['ID','Lat','Lon']]
    g=aqlocs.merge(locs,how='left',left_on='LocationID',right_on='ID')
    g["lat"] = np.floor(g["Lat"] * 1000).astype(int,errors='ignore')
    g["lon"] = np.floor(g["Lon"] * 1000).astype(int,errors='ignore')

    aqtiles = g.groupby(['lat','lon']).agg(list).reset_index()
    COLS_2_AGG_BY_MEDIAN = ['Vn.PM10_med_2020','Vn.PM10_med_2021','Vn.PM10_med_2022']
    COLS_2_AGG_BY_MEAN = ['Vn.PM10_avg_2020','Vn.PM10_avg_2021','Vn.PM10_avg_2022']
    COLS_2_AGG_BY_MAX = ['Vn.PM10_max_2020','Vn.PM10_max_2021','Vn.PM10_max_2022']
    ##COLS_2_DROP = list(set(aqtiles.columns) - {'lat','lon'} - set(COLS_2_AGG_BY_MEDIAN) - set(COLS_2_AGG_BY_MAX))
    COLS_2_DROP = list(set(aqtiles.columns) - {'lat','lon'} - set(COLS_2_AGG_BY_MEAN))
    for c in COLS_2_AGG_BY_MEDIAN:
        aqtiles[c] = [stat.median(l) for l in aqtiles[c]]
    for c in COLS_2_AGG_BY_MEAN:
        aqtiles[c] = [stat.mean(l) for l in aqtiles[c]]
    for c in COLS_2_AGG_BY_MAX:
        aqtiles[c] = [max(l) for l in aqtiles[c]]
    aqtiles.drop(COLS_2_DROP,axis=1,inplace=True)
    aqtiles['lat']=aqtiles['lat'].astype('Int64')
    aqtiles['lon']=aqtiles['lon'].astype('Int64')

    #merge met df op lat en lon
    df = pd.merge(df, aqtiles, on = ['lat','lon'], how='left')
    return df 

#### 1.5 Green and Grey information from ODE

In [39]:
def compute_intersections_with_tiles(df, polygon, col):
 ###   print("Polygon ", polygon.bounds)
    min_lat, min_lon, max_lat, max_lon = polygon.bounds
    min_lon = floor3(max(min_lon-TILESIZE_LON,GRID_LEFT_LON))
    min_lat = floor3(max(min_lat-TILESIZE_LAT,GRID_BOT_LAT))
    max_lon = floor3(min(max_lon+TILESIZE_LON,GRID_RIGHT_LON-0.001))
    max_lat = floor3(min(max_lat+TILESIZE_LAT,GRID_TOP_LAT-0.001))
  ###  print(min_lat,max_lat,min_lon,max_lon)
    ntiles=0
    ta=0
    for lat in np.arange(min_lat, max_lat, TILESIZE_LAT):
        for lon in np.arange(min_lon,max_lon, TILESIZE_LON):
            
            latlng_coords = [(lat,lon), (lat+TILESIZE_LAT,lon), (lat+TILESIZE_LAT,lon+TILESIZE_LON), (lat,lon+TILESIZE_LON)]
            tile = Polygon(latlng_coords)
           ### print("Tile: ",tile)
            intersection = polygon.intersection(tile)
            int_area = min(intersection.area,polygon.area,tile.area) # <-- to prevent tiny area estimation errors

            if int_area > 0:
                try:
                    ntiles=ntiles+1
                    a = df.loc[(int(1000*lat), int(1000*lon)),col]
                    df.loc[(int(1000*lat), int(1000*lon)),col] =  a + int_area
                    ta=ta+int_area
                except:
                    print("ERROR IN INTERSECTION of Polygon: ",polygon,"\nand Tile ",tile,"\ndf-index is ", int(1000*lat), int(1000*lon))
                    pass   ### TODO: fix this a different way! sometimes the lon or lat get out of bounds
    return ntiles,ta

#########################################
def ReverseLatLng(p):
    ###print("reversing ",p)
    coords = p.exterior.coords[:]
    new_coords = [(y, x) for x, y in coords]
    return Polygon(new_coords)

def GeoShapes2Polygons(lgs):
    lp = []
    for i in range(0,len(lgs)):
        try:
            js = json.loads(lgs[i])
            if (js['type'] == 'Polygon'):
                lp.append(ReverseLatLng(Polygon(js['coordinates'][0])))
            elif (js['type'] == 'MultiPolygon'):
                mp=shape(js)
                ###for p in mp.geoms:
                ###    print(mapping(p),type(mapping(p)))
                for geometry in mp.geoms:
                    lp.append(ReverseLatLng(Polygon(geometry)))
            else:
                print("it is a ",js['type'])
        except:
            print("Shape ",i, ": ERR ",lgs[i])
            pass
    return lp


def addShapeIntersection(df,col,ssh):
    # make 
    df[col] = 0 
    df.set_index(['lat','lon'],inplace=True)


    # only polygons..abs
    print(len(ssh), " geoshapes")
    ssh = GeoShapes2Polygons(ssh)
    print(len(ssh), " polygons")

    # loop through the list of shapes, mark the intersecting tiles
    for i in range(0,len(ssh)):
        try:
            ###print("geo_shape ",i," : ", ssh[i])
            #json_obj = json.loads(ssh[i])
            ###print("Json ",i," : ", json_obj)
            #bpoly = Polygon(json_obj['coordinates'][0])
            ###print("Shape ",i," : ", bpoly)
            bpoly=ssh[i]
            ntiles, totalarea = compute_intersections_with_tiles(df,bpoly,col)
            if (floor3(totalarea )> floor3(bpoly.area)):
            ####if (ntiles>0):
                print("Shape ",i,bpoly," \n--------- intersects ", ntiles, "tiles. Intarea: ",totalarea," out of ",bpoly.area)
        except:
            print("Shape ",i, ": ERR ",ssh[i])
            pass
    df.reset_index(inplace=True)

    # transform the area sum to percentage
    tilearea = TILESIZE_LAT * TILESIZE_LON
    ## print("Tilearea in degrees: ",tilearea)
    df[col] = df[col] / tilearea
    return df



#### 1.6 Bioclim variables

In [40]:
# temperature and humidity information from WORLDCLIM
# TODO: extract the 30s resolution data, now it is 5 m because of timeout (hardware limitation?)

def getBioclimVars(aggform,coltemp,colprec):
    
    import os
    import subprocess
    import sys
    os.environ['LATLONRES'] = "5m" # visible in this process + all children
    from latlon_utils import get_climate
    
    lat_list = []
    lon_list = []
    coltemp_list=[]
    colprec_list=[]

    for lat10x in np.arange(GRID_BOT_LAT,GRID_TOP_LAT,TILESIZE_LAT*10):
        for lon10x in np.arange(GRID_LEFT_LON,GRID_RIGHT_LON,TILESIZE_LON*10):
            tp = get_climate(lat10x + TILESIZE_LAT*5, lon10x + TILESIZE_LON*5)
            tpt = tp[('tavg',aggform)]
            tpp = tp[('prec',aggform)]
            for lat in np.arange(lat10x,lat10x+TILESIZE_LAT*10,TILESIZE_LAT):
                for lon in np.arange(lon10x,lon10x+TILESIZE_LON*10,TILESIZE_LON):
                    lat_list.append(lat)
                    lon_list.append(lon)
                    coltemp_list.append(tpt)
                    colprec_list.append(tpp)
        print(lat10x)

    dftp = pd.DataFrame({'lat': lat_list, 'lon': lon_list, coltemp:coltemp_list,colprec:colprec_list})
    dftp['lat'] = [int(np.floor(x * 1000)) for x in dftp['lat']]
    dftp['lon'] = [int(np.floor(x * 1000)) for x in dftp['lon']]
    dftp.drop_duplicates(subset=['lat', 'lon'], inplace=True)
    return dftp

In [41]:
def addWorldClim(df,aggform,coltemp,colprec):
    dftp = getBioclimVars(aggform,coltemp,colprec)
    #merge met df op lat en lon
    df = pd.merge(df, dftp, on = ['lat','lon'], how='left')
    return df

#### 1.7 Generate the codebook

In [42]:
ExplanationDict = {'lat':'latitude of the bottom-left tile corner x 1000', 
                     'lon':'longitude of the bottom-left tile corner x 1000', 
                   'Enb.avg_income':'the average income per working individual, in this neighbourhood', 
                   'Enb.woz':'average house value', 
                   'Enbp.high_education':'percentage of people with high education',
                   'Enbp.independent':'percentage of people working', 
                   'Enbp.low_education':'percentage of people with low education', 
                   'Enbp.unemployed':'unemployment percentage',
       'Gcb.buurt':'the name of the neighbourhood where this tile belongs to', 
       'Gcb.buurtcode':'the unique identifier code of the neighbourhood', 
       'Pnb.cohesion':'perceived social cohesion score', 
       'Pnb.diversity':'ethnical diversity score',
       'Pnb.good_life':'score for perceived good life', 
       'Pnb.pop':'number of residents/tile in this neigbourhood', 
       'Pnbp.complains_aq':'percentage of people compaining about air quality', 
       'Pnbp.complains_noise':'percentage of people compaining about noise',
       'Pnp.complains_noise':'number of omplaints about noise',
       'Pnbp.feels_unsafe':'percentage of people that feel unsafe', 
       'Pnbp.mostly_by_car':'percentage of people that move mostly by car', 
       'Pnbp.mostly_walking':'percentage of people that move mostly by walking',
       'Pnbp.unhappy':'percentage of people that feel unhappy', 
       'Vc.milieuzone':'can trucks drive through (True/False)', 
       'Vc.mode_tree_species':'the most common tree species in this tile',
       'Vn.number_wild_species':'number of different observed plant or animal species in this tile', 
       'Vc.mode_species':'the most observed plant or animal species',
       'Vn.PM10_avg_2020':'the average PM10 value of all the measurements in 2020', 
       'Vn.PM10_avg_2021':'',
       'Vn.PM10_avg_2022':'', 
       'Vn.PM10_med_2020':'the median PM10 value of all the measurements in 2020, on the sensors of this tile', 
       'Vn.PM10_med_2021':'the median PM10 value of all the measurements in 2021, on the sensors of this tile',
       'Vn.PM10_med_2022':'the median PM10 value of all the measurements in 2022, on the sensors of this tile', 
       'Vn.med_tree_height':'the median height of the trees on this tile', 
       'Vn.number_species':'the number of plant, animals, birds and insects species observed on this tile',
       'Vn.number_tree_species':'the number of different tree species', 
       'Vn.number_trees':'the total number of trees on this tile', 
       'Vn.prec_ann':'the average rainfall at the centerpoint of this tile in 2022',
       'Vn.tavg_ann':'the average temperature at the centerpoint of this tile in 2022', 
       'Vnp.Green':'the percentage of public green space', 
       'Inp.Grey':'the percentage of streets and pavements',
       'Vnp.Grey':''
}


In [43]:

EIC = "EIC"
ODE = "ODE"
GBIF = "GBIF"
TNO = "TNO"
BIOCLIM = "WORLDCLIM"

SourceDict = {'lat':'', 'lon':'', 
      'Enb.avg_income':EIC, 
      'Enb.woz':EIC,
      'Enbp.high_education':EIC,
      'Enbp.independent':EIC,
      'Enbp.low_education':EIC,
      'Enbp.unemployed':EIC,
      'Gcb.buurt':ODE, 
      'Gcb.buurtcode':ODE,
      'Pnb.cohesion':EIC,
      'Pnb.diversity':EIC,
      'Pnb.good_life':EIC,
      'Pnb.pop':EIC,
      'Pnbp.complains_aq':EIC, 
      'Pnbp.complains_noise':EIC,
      'Pnbp.feels_unsafe':EIC,
      'Pnbp.mostly_by_car':EIC,
       'Pnbp.mostly_walking':EIC,
       'Pnbp.unhappy':EIC,
       'Vc.milieuzone':ODE,
       'Vc.mode_tree_species':ODE,
       'Vn.number_wild_species':GBIF, 
       'Vc.mode_species':GBIF,
       'Vn.PM10_avg_2020':TNO, 
       'Vn.PM10_avg_2021':TNO,
       'Vn.PM10_avg_2022':TNO, 
       'Vn.PM10_med_2020':TNO, 
       'Vn.PM10_med_2021':TNO,
       'Vn.PM10_med_2022':TNO,
       'Vn.med_tree_height':ODE + "- bomen", 
       'Vn.number_species': GBIF,
       'Vn.number_tree_species': ODE + "- bomen",  
       'Vn.number_trees': ODE+ "- bomen",  
       'Vn.prec_ann':BIOCLIM,
       'Vn.tavg_ann':BIOCLIM, 
       'Vnp.Green':ODE + "- openbaar groen", 
       'Inp.Grey':ODE + "- bestrating",
       'Vnp.Grey':''
}


In [44]:

UnitDict = {'lat':'latitude degree ', 
            'lon':'longitude degree', 
      'Enb.avg_income':'euro', 
      'Enb.woz':'thousand euro',
      'Enbp.high_education':'%',
      'Enbp.independent':'%',
      'Enbp.low_education':'%',
      'Enbp.unemployed':'%',
      'Gcb.buurt':'', 
      'Gcb.buurtcode':'',
      'Pnb.cohesion':'score from 1 to 10',
      'Pnb.diversity':'computed index between 0 and 1',
      'Pnb.good_life':'score from 1 to 10',
      'Pnb.pop':'residents',
      'Pnbp.complains_aq':'%', 
      'Pnbp.complains_noise':'%',
      'Pnbp.feels_unsafe':'%',
      'Pnbp.mostly_by_car':'%',
       'Pnbp.mostly_walking':'%',
       'Pnbp.unhappy':'%',
       'Vc.milieuzone':'True/False',
       'Vc.mode_tree_species':'',
       'Vn.number_wild_species':'species', 
       'Vc.mode_species':'',
       'Vn.PM10_avg_2020':'$\mu/m^3$', 
       'Vn.PM10_avg_2021':'$\mu/m^3$',
       'Vn.PM10_avg_2022':'$\mu/m^3$', 
       'Vn.PM10_med_2020':'$\mu/m^3$', 
       'Vn.PM10_med_2021':'$\mu/m^3$',
       'Vn.PM10_med_2022':'$\mu/m^3$', 
       'Vn.med_tree_height':'m', 
       'Vn.number_species': 'species',
       'Vn.number_tree_species': 'species',  
       'Vn.number_trees': 'trees',  
       'Vn.prec_ann':'mm',
       'Vn.tavg_ann':'$\degree C$', 
       'Vnp.Green':'%', 
       'Inp.Grey':'%',
       'Vnp.Grey':''
}

In [45]:
def prefixContains(varname,caracter):
    return (caracter in varname.split(".")[0])

# TODO: separate the scale functions from the print
def scaleTo01(l):
    vmax = np.max(l) #TODO: fix!
    if (vmax>10): # very bad heuristics to determine if a variable is in 0-100 or 0-1 scale
        l = l / 100
        vmax = np.max(l)
    if (vmax < 1.0):
        return l
    #print("SCALE.. vmax = ", vmax)
    l01 = l.copy()
    for i in range(len(l)):
        if (not np.isnan(l[i])):
            if l[i] >=0:
                l01[i] = np.floor(l[i]*100/vmax)/100 
                #print("! ", l[i],l01[i])
            else: 
                #print("? ", l[i])
                l01[i] = np.nan
    return l01

def scaleTo100(l,maxp):
    vmax = np.max(l) #TODO: fix!
    if (vmax < 10): # very bad heuristics to determine if a variable is in 0-100 or 0-1 scale
        l = [x*100 for x in l]
    if (vmax <= maxp):
        return l
    l100 = l.copy()
    for i in range(len(l)):
        if (not np.isnan(l[i])):
            if (l[i] >=0) and (l[i] <=maxp):
                l100[i] = l[i]
            else: 
                l100[i] = np.nan
    return l100

def string2Float_older(l):
    GARBAGE_CHARS = {'.','-'} 
    lf = [np.nan if ((v=='.') or (v=='-')) else v for v in l]
    lf = [v.replace(',','.') if isinstance(v,str) else v for v in lf]
    lf = [(np.nan if (v == np.nan) else np.floor(float(v)*100)/100)  for v in lf]
    return lf

def string2Float(l):
    lf = l.copy()
    for i in range(len(l)):
        if (not (l[i] is None)):
            try:
                lf[i] = l[i].replace(',','.')
                lf[i] = np.floor(float(lf[i])*100)/100
                ##print("! ", l[i],l01[i])
            except: 
                ###print("? ", l[i])
                lf[i] = np.nan
    return lf

def getRange(l,lname):
    try:
        if prefixContains(lname,'c'):
            vals = set(l) - {np.nan}
            mi = min(vals)
            ma = max(vals)
        else:
            mi = np.floor(np.min(l)*100)/100
            ma = np.floor(np.max(l)*100)/100
    except:
        ###print("{} - ERR".format(__name__))
        mi=""
        ma=""
    return mi,ma


def printGridInMarkdown(df):
    print("| Variable | Explanation | Unit | Range | Source |")
    print("| :--- | :--- | :--- | :--- | :--- | ")

    for x in df.columns:
        ###print(x,df[x].dtype)
        if (prefixContains(x,'n')) and (df[x].dtype=='object'):
            df[x] = string2Float(df[x])
            ###print("s2f --",x,df[x].count())
        if (prefixContains(x,'p')): # percent
            ###df[x] = scaleTo01(df[x])
            df[x] = scaleTo100(df[x],90)
            ###print("p -- ",x,df[x].count())
        
        mi,ma = getRange(df[x],x)
        if prefixContains(x,'b'):
            print("| *{0}* | {3} | {4}|{1} - {2}| {5}| ".format(x,mi,ma,ExplanationDict[x],UnitDict[x],SourceDict[x]))
        else:
            print("| **{0}** | {3} | {4}|{1} - {2}| {5}|".format(x,mi,ma,ExplanationDict[x],UnitDict[x], SourceDict[x]))
    return

def printGridInLatex(df):
    print("Variable & Explanation & Unit & Range & Source \\")
    print("\hline ")

    for x in df.columns:
        ###print(x,df[x].dtype)
        if (prefixContains(x,'n')) and (df[x].dtype=='object'):
            df[x] = string2Float(df[x])
            ###print("s2f --",x,df[x].count())
        if (prefixContains(x,'p')): # percent
            ###df[x] = scaleTo01(df[x])
            df[x] = scaleTo100(df[x],90)
            ###print("p -- ",x,df[x].count())
        
        mi,ma = getRange(df[x],x)
        if prefixContains(x,'b'):
            print(" \{\\em {0} \} & {3} & {4}|{1} - {2}& {5}\\\\ ".format(x,mi,ma,ExplanationDict[x],UnitDict[x],SourceDict[x]))
        else:
            print(" \{\\bf {0} \} & {3} & {4}|{1} - {2}& {5}\\\\".format(x,mi,ma,ExplanationDict[x],UnitDict[x], SourceDict[x]))
    return

In [46]:
#d = df[["Enb.avg_income","Enb.woz"]]
#d.sample(5)
#df = dfsaved.copy()
#he = scaleTo01(string2Float(df["Enbp.high_education"]))
#np.max(he)

## 2. ACTION

#### 2.1 Create the grid by reading in the data sources and processing them one by one

In [47]:
# SETTINGS
# datasets from Open Data Eindhoven
ode_folder = "data/ode-downloads-mar-2023/"

# TODO: add "ode-downloads-mar-2023/meldingen-openbare-ruimte.csv"

ALLVARS=[]

# empty grid
df = createEmptyGrid()
print("Created the empty grid \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

Created the empty grid 
 (20000, 2) {'lat', 'lon'}


In [48]:

# assign each tile to a neighbourhoud
dfb = pd.read_csv(ode_folder + "buurten.csv",sep=';') [['Buurtcode ', 'Buurtnaam ', 'geo_shape']]
df = addBuurten(df,dfb)
print("Added buurten \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns


Added buurten 
 (20000, 4) {'Gcb.buurt', 'Gcb.buurtcode'}


In [49]:

# add demographic information
dfpop = pd.read_csv(ode_folder + "buurt_bevolking.csv",sep=";") [['Buurtcode ','number of residents']]
df = addPopulationDensity(df,dfpop)
print("Added population info \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

# add more info
dfbi = pd.read_csv(ode_folder + "Buurtgegevens - Buurten.csv",sep=";")
df = addBuurtStuff(df,dfbi)
print("Added Buurtgegevens \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

Added population info 
 (20000, 5) {'Pnb.pop'}
Added Buurtgegevens 
 (20000, 20) {'Enb.woz', 'Pnbp.mostly_by_car', 'Enbp.high_education', 'Pnbp.unhappy', 'Enbp.low_education', 'Pnbp.complains_noise', 'Pnbp.mostly_walking', 'Pnbp.feels_unsafe', 'Pnb.cohesion', 'Enb.avg_income', 'Pnb.good_life', 'Enbp.independent', 'Pnb.diversity', 'Pnbp.complains_aq', 'Enbp.unemployed'}


In [50]:
# tree info
dfb = pd.read_csv(ode_folder + "bomen.csv",quotechar='"',sep=';') 
                              # 'geo_point_2d', 'HOOGTE'
COLUMNS_TO_DROP = ['OBJECTID', 'BOOMNUMMER', 'GEOVISIA_ID', 'EIGENAAR', 'BEHEERDER',
      'STATUS_TER_INDICATIE', 'BOOM_DEF_AFWEZIG', 'EINDBEELD', 'BOOMSOORT_VARIETEIT',
      'PLANTWIJZE', 'BOOMROOSTER', 'VERLICHTING', 'EPR_AANWEZIG',
      'EPR_PREVENTIEF_INSPECTEREN', 'EPR_BESTRIJDINGSMETHODE', 'EPR_STADIUM',
      'EPR_DATUM_CONSTATERING', 'EPR_RISICOPROFIEL',
      'EPR_DATUM_MECH_BESTREDEN', 'EPR_DATUM_1_BIO_BESTRIJDING',
      'EPR_DATUM_2_BIO_BESTRIJDING', 'NAZORGBOOM', 'PROJECTNAAM', 'geo_shape']
dfb.drop(COLUMNS_TO_DROP,axis=1,inplace=True)
df = addTreeInfo(df, dfb)
print("Added Trees \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

# gbif info
file_gbif = "./data/gbif/0151022-230224095556074.csv"
dfobs = pd.read_csv(file_gbif,sep="\t")[['species','decimalLatitude', 'decimalLongitude', 'year']]
df = addBiodiversityInfo(df,dfobs)
print("Added gbif information \n", df.shape, set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

Added Trees 
 (20000, 24) {'Vn.med_tree_height', 'Vc.mode_tree_species', 'Vn.number_trees', 'Vn.number_tree_species'}
__main__: 
Added gbif information 
 (20000, 26) {'Vn.number_species', 'Vc.mode_species'}


In [51]:
# openbaar groen
dfg = pd.read_csv(ode_folder + "openbaar-groen0.csv",sep=";")[['geo_shape']]
dfg = dfg.dropna().reset_index()
df = addShapeIntersection(df,"Vnp.Green",dfg["geo_shape"])
print("Added public green info ", df.shape, "\n", set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

45588  geoshapes
45604  polygons
Added public green info  (20000, 27) 
 {'Vnp.Green'}


In [52]:

# openbaar grijs
dfg = pd.read_csv(ode_folder + "bestrating.csv",sep=";")[['geo_shape']]
dfg = dfg.dropna().reset_index()
df = addShapeIntersection(df,"Inp.Grey",dfg["geo_shape"])
print("Added street/stone info ", df.shape, "\n", set(df.columns)-set(ALLVARS))
ALLVARS = df.columns


102792  geoshapes
102793  polygons
Added street/stone info  (20000, 28) 
 {'Inp.Grey'}


In [53]:

# milieuzones
sfg = pd.read_csv(ode_folder + "milieuzone.csv",sep=";")['Geo Shape']
df = addShapeIntersection(df,"Vc.milieuzone",sfg)
df["Vc.milieuzone"] = [1 if x > 0.5 else 0 for x in df["Vc.milieuzone"]]
print("Added milieuzone 1/0 ", df.shape, "\n", set(df.columns)-set(ALLVARS))
ALLVARS = df.columns

2  geoshapes
2  polygons
Added milieuzone 1/0  (20000, 29) 
 {'Vc.milieuzone'}


In [54]:
# air quality 
aq = pd.read_csv(r"./data/AQ-2019-tm-feb2023-by-Skyler/full_measurement.csv",quotechar='"',sep=',')
aqlocs = agg_AQ_per_Location(aq) # <-- duurt 5+ min 
locs=pd.read_csv(r"./data/AQ-2019-tm-feb2023-by-Skyler/locaties-airboxen.csv",quotechar='"',sep=';')
df = agg_AQ_per_Tile(df,aqlocs,locs)
print("Added aggregated AQ sensor data ", df.shape, "\n", set(df.columns)-set(ALLVARS))
ALLVARS = df.columns
df.to_csv("ehv-grid-mar23-without-bioclim.csv",index=False)

Added aggregated AQ sensor data  (20000, 32) 
 {'Vn.PM10_avg_2022', 'Vn.PM10_avg_2020', 'Vn.PM10_avg_2021'}


In [55]:
# bioclim
df = addWorldClim(df,'ann','Vn.tavg_ann','Vn.prec_ann')
print("Added bioclim vars ", df.shape, "\n", set(df.columns)-set(ALLVARS))
ALLVARS = df.columns


51.4
51.41
51.419999999999995
51.42999999999999
51.43999999999999
51.44999999999999
51.45999999999999
51.469999999999985
51.47999999999998
51.48999999999998
51.49999999999998
Added bioclim vars  (20000, 34) 
 {'Vn.tavg_ann', 'Vn.prec_ann'}


#### 2.2 Export the datagrid as CSV and generate the MD documentation

In [56]:
# sort the columns, so that the categories are grouped
df_latlon = df.iloc[:, :2]
df_etc = df.iloc[:, 2:].sort_index(axis=1)
df = pd.concat([df_latlon, df_etc], axis=1)
df.head(10)

,lat,lon,Enb.avg_income,Enb.woz,Enbp.high_education,Enbp.independent,Enbp.low_education,Enbp.unemployed,Gcb.buurt,Gcb.buurtcode,...,Vn.PM10_avg_2020,Vn.PM10_avg_2021,Vn.PM10_avg_2022,Vn.med_tree_height,Vn.number_species,Vn.number_tree_species,Vn.number_trees,Vn.prec_ann,Vn.tavg_ann,Vnp.Green
0,51400,5350,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
1,51400,5351,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
2,51400,5352,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
3,51400,5353,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
4,51400,5354,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
5,51400,5355,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
6,51400,5356,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
7,51400,5357,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
8,51400,5358,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0
9,51400,5359,NaN,NaN,NaN,NaN,NaN,NaN,Buiten,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.25,9.924337,0.0


In [57]:
dfsaved=df.copy()
#df=dfsaved.copy()

In [58]:

# generate MD documentation
printGridInMarkdown(df)
#printGridInLatex(df)

| Variable | Explanation | Unit | Range | Source |
| :--- | :--- | :--- | :--- | :--- | 
| **lat** | latitude of the bottom-left tile corner x 1000 | latitude degree |51400.0 - 51499.0| |
| **lon** | longitude of the bottom-left tile corner x 1000 | longitude degree|5350.0 - 5549.0| |
| *Enb.avg_income* | the average income per working individual, in this neighbourhood | euro|9.6 - 150.19| EIC| 
| *Enb.woz* | average house value | thousand euro|111.0 - 923.0| EIC| 
| *Enbp.high_education* | percentage of people with high education | %|0.0 - 83.9| EIC| 
| *Enbp.independent* | percentage of people working | %|50.6 - 87.9| EIC| 
| *Enbp.low_education* | percentage of people with low education | %|2.6 - 83.3| EIC| 
| *Enbp.unemployed* | unemployment percentage | %|2.0 - 50.0| EIC| 
| *Gcb.buurt* | the name of the neighbourhood where this tile belongs to | |'t Hofke - Zwaanstraat| ODE| 
| *Gcb.buurtcode* | the unique identifier code of the neighbourhood | |0 - 733| ODE| 
| **Inp.Grey** | th

In [61]:
df = df[df['Gcb.buurt'] != "Buiten"]
df.shape

(11832, 34)

In [62]:

# save to CSV file

df.to_csv("ehv-grid-mei23.csv",index=False)